TensorFlow version: 2.12.0
Built with CUDA?    False
GPUs detected:      []
Found 17416 images belonging to 2 classes.
Found 4355 images belonging to 2 classes.
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (No

KeyboardInterrupt: 

In [1]:
import os
import glob
import traceback

# ─── FORCE CPU ────────────────────────────────────────────────────────────────
os.environ['CUDA_VISIBLE_DEVICES'] = ''

import numpy as np
import cv2
import gradio as gr
import tensorflow as tf
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint

# ─── DEVICE CHECK ─────────────────────────────────────────────────────────────
print("TensorFlow version:", tf.__version__)
print("Built with CUDA?   :", tf.test.is_built_with_cuda())
print("GPUs detected      :", tf.config.list_physical_devices('GPU'))

# ─── SETTINGS ─────────────────────────────────────────────────────────────────
# **Downscale for debugging**—bump these back up once you can complete epoch 1
IMG_SIZE      = (128, 128)
BATCH_SIZE    = 1
EPOCHS        = 10
LEARNING_RATE = 1e-4

TRAIN_DIR = "dataset/train"
VAL_DIR   = "dataset/val"
CHECKPOINT_DIR = "checkpoints"
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

# ─── DATA LOADERS ─────────────────────────────────────────────────────────────
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)
val_generator = ImageDataGenerator(rescale=1.0/255).flow_from_directory(
    VAL_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# ─── AUTO-DETECT LAST CHECKPOINT ──────────────────────────────────────────────
ckpts = sorted(glob.glob(os.path.join(CHECKPOINT_DIR, "model_epoch_*.h5")))
if ckpts:
    latest_ckpt = ckpts[-1]
    last_epoch = int(os.path.basename(latest_ckpt).split("_")[-1].split(".")[0])
    print(f"Found checkpoint {latest_ckpt}, resuming from epoch {last_epoch + 1}")
    model = load_model(latest_ckpt)
    initial_epoch = last_epoch
else:
    print("No checkpoint found — starting from scratch")
    initial_epoch = 0

    # ─── BUILD & COMPILE MODEL ────────────────────────────────────────────────
    base_model = VGG19(
        weights='imagenet',
        include_top=False,
        input_tensor=Input(shape=(*IMG_SIZE, 3))
    )
    for layer in base_model.layers[:15]:
        layer.trainable = False

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.3)(x)
    outputs = Dense(2, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=outputs)
    model.compile(
        optimizer=Adam(learning_rate=LEARNING_RATE),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

model.summary()

# ─── QUICK SANITY CHECK ───────────────────────────────────────────────────────
print("\nTesting one batch with train_on_batch to isolate issues…")
try:
    x_batch, y_batch = next(train_generator)
    print("  batch shapes:", x_batch.shape, y_batch.shape)
    loss, acc = model.train_on_batch(x_batch, y_batch)
    print(f"  train_on_batch succeeded → loss={loss:.4f}, acc={acc:.4f}")
except Exception:
    print("Error during train_on_batch:")
    traceback.print_exc()
    raise SystemExit("Fix the above error before proceeding to fit()")

# ─── CALLBACKS ────────────────────────────────────────────────────────────────
csv_logger    = CSVLogger('training_log.csv', append=(initial_epoch > 0))
checkpoint_cb = ModelCheckpoint(
    filepath=os.path.join(CHECKPOINT_DIR, "model_epoch_{epoch:02d}.h5"),
    save_weights_only=False,
    save_freq='epoch'
)

# ─── TRAINING (with full traceback) ───────────────────────────────────────────
print(f"\nStarting training from epoch {initial_epoch + 1} (IMG_SIZE={IMG_SIZE}, BATCH_SIZE={BATCH_SIZE})…")
try:
    model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=EPOCHS,
        initial_epoch=initial_epoch,
        callbacks=[csv_logger, checkpoint_cb],
        workers=0,
        use_multiprocessing=False
    )
except Exception:
    print("Error during model.fit():")
    traceback.print_exc()
    raise SystemExit("Fix the above error or reduce IMG_SIZE/BATCH_SIZE further")

# ─── SAVE FINAL MODEL ─────────────────────────────────────────────────────────
print("\nSaving final model…")
model.save("deepfake_detector_tf")
model.save("deepfake_detector.h5")

# ─── EVALUATE ─────────────────────────────────────────────────────────────────
loss, acc = model.evaluate(val_generator, verbose=1)
print(f"\nValidation Accuracy: {acc * 100:.2f}%")

# ─── GRADIO INTERFACE ────────────────────────────────────────────────────────
class_names = list(train_generator.class_indices.keys())
def classify_image(img_np):
    img = cv2.resize(img_np, IMG_SIZE)
    img = img.astype(np.float32) / 255.0
    preds = model.predict(img[None, ...], verbose=0)[0]
    return {class_names[i]: float(preds[i]) for i in range(len(class_names))}

gr.Interface(
    fn=classify_image,
    inputs=gr.Image(type="numpy", label="Upload Image"),
    outputs=gr.Label(num_top_classes=2),
    title="Deepfake Frame Classifier (CPU Only)",
    description="Upload a frame to classify it as real or fake."
).launch()


TensorFlow version: 2.12.0
Built with CUDA?   : False
GPUs detected      : []
Found 17416 images belonging to 2 classes.
Found 4355 images belonging to 2 classes.
No checkpoint found — starting from scratch
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 128, 128, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 128, 128, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 64, 64, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 64, 64, 128)       73856     
                                                    

: 